# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

**Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **A**: 10.23433234744176
- **B**: 7.892332347441762
- **C**: 8.382332347441762 ✅
- **D**: 9.123332347441762

**Question 2: What is the 13th number yielded**
- **A**: 4.236551275463989
- **B**: 3.605551275463989 ✅
- **C**: 2.345551275463989
- **D**: 5.678551275463989

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [4]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

summation = 0
for sqrt_value in generator:
    summation += sqrt_value

print(summation)
#######################################
for result in square_root_generator(13):
    continue
print(result)

8.382332347441762
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**
- **A**: 353 ✅
- **B**: 365
- **C**: 378
- **D**: 390




In [32]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [33]:
import dlt
import duckdb

In [34]:
pipeline = dlt.pipeline(destination="duckdb", dataset_name="people", credentials="data.db")

/Users/shayan/miniconda3/envs/py10/lib/python3.10/site-packages/dlt/pipeline/__init__.py:108: Dlt04DeprecationWarning: The `credentials argument` to pipeline is deprecated and will be removed in a future version. Pass the same credentials to the `destination` instance instead, e.g. pipeline(destination=dlt.destinations.duckdb(credentials=...)). Deprecated in dlt 0.4.0 to be removed in 0.5.0.
  credentials_argument_deprecated("pipeline", credentials, destination)


In [35]:
info = pipeline.run(people_1(), table_name="people", write_disposition="replace")
print(info)

2024-02-12 11:06:36,498|[WARNING              ]|79021|8017170432|dlt|configuration.py|_path_from_pipeline:180|Duckdb attached to pipeline dlt_ipykernel_launcher in path data.db was deleted. Attaching to duckdb database 'dlt_ipykernel_launcher.duckdb' in current folder.
2024-02-12 11:06:36,869|[WARNING              ]|79021|8017170432|dlt|configuration.py|_path_from_pipeline:180|Duckdb attached to pipeline dlt_ipykernel_launcher in path data.db was deleted. Attaching to duckdb database 'dlt_ipykernel_launcher.duckdb' in current folder.


Pipeline dlt_ipykernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/shayan/personal/DE-zoomcamp-playground/workshop1/data.db location to store data
Load package 1707723396.526774 is LOADED and contains no failed jobs


In [36]:
print(pipeline.dataset_name)
conn = duckdb.connect("data.db")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_1            │
│ people_with_id      │
└─────────────────────┘

In [37]:
conn.sql("SELECT sum(age) FROM people")

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘

In [38]:
info = pipeline.run(people_2(), table_name="people", write_disposition="append")

2024-02-12 11:07:03,644|[WARNING              ]|79021|8017170432|dlt|configuration.py|_path_from_pipeline:180|Duckdb attached to pipeline dlt_ipykernel_launcher in path data.db was deleted. Attaching to duckdb database 'dlt_ipykernel_launcher.duckdb' in current folder.


In [39]:
conn.sql("SELECT sum(age) FROM people")

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**
- **A**: 215
- **B**: 266 ✅
- **C**: 241
- **D**: 258

In [40]:
info = pipeline.run(people_1(), table_name="people_with_id", write_disposition="replace", primary_key="id")
conn.sql('select * from people_with_id;')

2024-02-12 11:07:20,803|[WARNING              ]|79021|8017170432|dlt|configuration.py|_path_from_pipeline:180|Duckdb attached to pipeline dlt_ipykernel_launcher in path data.db was deleted. Attaching to duckdb database 'dlt_ipykernel_launcher.duckdb' in current folder.


┌───────┬──────────┬───────┬─────────┬───────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │   _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar      │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼───────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707723440.459401 │ F2RYTfijGmFRFA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707723440.459401 │ PEP+6ABW2o1Gaw │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707723440.459401 │ bHrPSUS95OyRJQ │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707723440.459401 │ /iuk5Rx1KjB4/Q │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707723440.459401 │ +afaX5YZs7jSmg │ NULL       │
└───────┴──────────┴───────┴─────────┴───────────────────┴────────────────┴────────────┘

In [41]:
info = pipeline.run(people_2(), table_name="people_with_id", write_disposition="merge", primary_key="id")
conn.sql('select * from people_with_id;')

2024-02-12 11:07:23,612|[WARNING              ]|79021|8017170432|dlt|configuration.py|_path_from_pipeline:180|Duckdb attached to pipeline dlt_ipykernel_launcher in path data.db was deleted. Attaching to duckdb database 'dlt_ipykernel_launcher.duckdb' in current folder.


┌───────┬──────────┬───────┬─────────┬───────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │   _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar      │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼───────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707723440.459401 │ F2RYTfijGmFRFA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707723440.459401 │ PEP+6ABW2o1Gaw │ NULL       │
│     5 │ Person_5 │    35 │ City_B  │ 1707723443.269835 │ O5T5Qdnoog/tQA │ Job_5      │
│     4 │ Person_4 │    34 │ City_B  │ 1707723443.269835 │ ULT9BQveHoicQg │ Job_4      │
│     8 │ Person_8 │    38 │ City_B  │ 1707723443.269835 │ qkyKZ/ehGPPjCg │ Job_8      │
│     7 │ Person_7 │    37 │ City_B  │ 1707723443.269835 │ 8t6yFHpTlF3kzA │ Job_7      │
│     3 │ Person_3 │    33 │ City_B  │ 1707723443.269835 │ E7njcMx8ukqX2g │ Job_3      │
│     6 │ Person_6 │ 

In [42]:
conn.sql("SELECT sum(age) FROM people_with_id")

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘